In [1]:
import pandas as pd

In [2]:
! wget https://s3.amazonaws.com/imcbucket/data/flights/2008.csv

--2022-01-29 15:24:54--  https://s3.amazonaws.com/imcbucket/data/flights/2008.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.238.189
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.238.189|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 689413344 (657M) [binary/octet-stream]
Saving to: ‘2008.csv’

2008.csv            100%[===================>] 657.48M  43.8MB/s    in 16s     

2022-01-29 15:25:10 (41.2 MB/s) - ‘2008.csv’ saved [689413344/689413344]



In [3]:
! wc -l 2008.csv

7009729 2008.csv


In [4]:
#f = "2008.csv"

In [5]:
#num_lines = sum(1 for l in open(f))

In [6]:
#size = int(num_lines / 100)

In [7]:
#import random
#skip_idx = random.sample(range(1, num_lines), num_lines - size)

In [8]:
#raw_pd = pd.read_csv(f, skiprows=skip_idx)

In [9]:
raw_pd = pd.read_csv("2008.csv")

In [10]:
raw_pd.head(5)

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2008,1,3,4,2003.0,1955,2211.0,2225,WN,335,N712SW,128.0,150.0,116.0,-14.0,8.0,IAD,TPA,810,4.0,8.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
1,2008,1,3,4,754.0,735,1002.0,1000,WN,3231,N772SW,128.0,145.0,113.0,2.0,19.0,IAD,TPA,810,5.0,10.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
2,2008,1,3,4,628.0,620,804.0,750,WN,448,N428WN,96.0,90.0,76.0,14.0,8.0,IND,BWI,515,3.0,17.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
3,2008,1,3,4,926.0,930,1054.0,1100,WN,1746,N612SW,88.0,90.0,78.0,-6.0,-4.0,IND,BWI,515,3.0,7.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
4,2008,1,3,4,1829.0,1755,1959.0,1925,WN,3920,N464WN,90.0,90.0,77.0,34.0,34.0,IND,BWI,515,3.0,10.0,0,NaN,0,2.0,0.0,0.0,0.0,32.0


In [11]:
raw_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7009728 entries, 0 to 7009727
Data columns (total 29 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Year               int64  
 1   Month              int64  
 2   DayofMonth         int64  
 3   DayOfWeek          int64  
 4   DepTime            float64
 5   CRSDepTime         int64  
 6   ArrTime            float64
 7   CRSArrTime         int64  
 8   UniqueCarrier      object 
 9   FlightNum          int64  
 10  TailNum            object 
 11  ActualElapsedTime  float64
 12  CRSElapsedTime     float64
 13  AirTime            float64
 14  ArrDelay           float64
 15  DepDelay           float64
 16  Origin             object 
 17  Dest               object 
 18  Distance           int64  
 19  TaxiIn             float64
 20  TaxiOut            float64
 21  Cancelled          int64  
 22  CancellationCode   object 
 23  Diverted           int64  
 24  CarrierDelay       float64
 25  WeatherDelay      

In [12]:
def t_timeperiod(origin):
    if origin is None:
        period = None
    elif origin > 0 and origin < 600:
        period = '00.01-05.59'
    elif origin >= 600 and origin <=1200:
        period = '06.00-11.59'
    elif origin >= 1200 and origin <= 1800:
        period = '12.00-17.59'
    elif origin >= 1800 and origin <= 2400:
        period = '18.00-24.00'
    else:
        period = 'NA'
    return period

In [13]:
raw_pd['DepTime'] = raw_pd['DepTime'].apply(lambda x:t_timeperiod(x))

In [17]:
max_distance = raw_pd['Distance'].max()
min_distance = raw_pd['Distance'].min()

In [19]:
max_ArrDelay = raw_pd['ArrDelay'].max()
min_ArrDelay = raw_pd['ArrDelay'].min()

In [21]:
def t_normalized_distance(origin):
    if origin is None:
        return None
    else:
        return ((origin-min_distance)/(max_distance-min_distance))

In [20]:
def t_normalized_ArrDelay(origin):
    if origin is None:
        return None
    else:
        return ((origin-min_ArrDelay)/(max_ArrDelay-min_ArrDelay))

In [22]:
raw_pd['Distance'] = raw_pd['Distance'].apply(lambda x:t_normalized_distance(x))

In [23]:
raw_pd['ArrDelay'] = raw_pd['ArrDelay'].apply(lambda x:t_normalized_ArrDelay(x))

In [25]:
features_pd = raw_pd[['UniqueCarrier','Origin','Dest',\
        'DepTime','TaxiOut','TaxiIn','DepDelay',\
        'DayOfWeek','Distance','ArrDelay']]

In [26]:
final_pd = features_pd.dropna()

In [27]:
final_pd.count()

UniqueCarrier    6855029
Origin           6855029
Dest             6855029
DepTime          6855029
TaxiOut          6855029
TaxiIn           6855029
DepDelay         6855029
DayOfWeek        6855029
Distance         6855029
ArrDelay         6855029
dtype: int64

In [29]:
features_pd.head(20)

,UniqueCarrier,Origin,Dest,DepTime,TaxiOut,TaxiIn,DepDelay,DayOfWeek,Distance,ArrDelay
0,WN,IAD,TPA,18.00-24.00,8.0,4.0,8.0,4,0.161382,0.169463
1,WN,IAD,TPA,06.00-11.59,10.0,5.0,19.0,4,0.161382,0.174832
2,WN,IND,BWI,06.00-11.59,17.0,3.0,8.0,4,0.101798,0.178859
3,WN,IND,BWI,06.00-11.59,7.0,3.0,-4.0,4,0.101798,0.172148
4,WN,IND,BWI,18.00-24.00,10.0,3.0,34.0,4,0.101798,0.185570
5,WN,IND,JAX,18.00-24.00,10.0,4.0,25.0,4,0.136740,0.177852
6,WN,IND,LAS,18.00-24.00,7.0,3.0,67.0,4,0.319127,0.193289
7,WN,IND,LAS,06.00-11.59,7.0,7.0,-1.0,4,0.319127,0.168121
8,WN,IND,MCI,06.00-11.59,19.0,6.0,2.0,4,0.088871,0.174832
9,WN,IND,MCI,12.00-17.59,6.0,3.0,0.0,4,0.088871,0.168792


In [30]:


#raw_pd.isnull().sum()
#nullCol_series = raw_pd.isnull().sum()
#notNull_list = list(nullCol_series[nullCol_series == 0].index)
#notNull_list
#requiredCol_list = ['TaxiIn','TaxiOut','ArrDelay']
#selectedCol_list = notNull_list + requiredCol_list
#noNull_pd = raw_pd.loc[:,selectedCol_list].dropna()
#noNull_pd.isnull().sum()
#noNull_pd.describe()
#noNull_pd.info()

In [31]:
#final_pd = noNull_pd

In [32]:
import pandas as pd
from seaborn import load_dataset
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression

In [33]:
train_df = final_pd.sample(frac=0.8,random_state=123)

In [34]:
test_df = final_pd.drop(train_df.index)

In [35]:
train_df.count()

UniqueCarrier    5484023
Origin           5484023
Dest             5484023
DepTime          5484023
TaxiOut          5484023
TaxiIn           5484023
DepDelay         5484023
DayOfWeek        5484023
Distance         5484023
ArrDelay         5484023
dtype: int64

In [36]:
test_df.count()

UniqueCarrier    1371006
Origin           1371006
Dest             1371006
DepTime          1371006
TaxiOut          1371006
TaxiIn           1371006
DepDelay         1371006
DayOfWeek        1371006
Distance         1371006
ArrDelay         1371006
dtype: int64

In [37]:
train_labels = train_df.filter(['ArrDelay'])
train_df = train_df.loc[:, train_df.columns != 'ArrDelay']

In [38]:
type(train_labels)

pandas.core.frame.DataFrame

In [39]:
test_labels = test_df.filter(['ArrDelay'])
test_df = test_df.loc[:, test_df.columns != 'ArrDelay']

In [40]:
X_train = train_df
X_train = train_df
y_train = train_labels
y_test = test_labels

In [41]:
categorical = list(X_train.select_dtypes('object').columns)
print(f"Categorical columns are: {categorical}")

numerical = list(X_train.select_dtypes('number').columns)
print(f"Numerical columns are: {numerical}")

Categorical columns are: ['UniqueCarrier', 'Origin', 'Dest', 'DepTime']
Numerical columns are: ['TaxiOut', 'TaxiIn', 'DepDelay', 'DayOfWeek', 'Distance']


In [42]:
from sklearn.tree import DecisionTreeClassifier, export_graphviz, DecisionTreeRegressor
from sklearn import tree
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [ ]:
# Define custom transformer
class ColumnSelector(BaseEstimator, TransformerMixin):
    """Select only specified columns."""
    def __init__(self, columns):
        self.columns = columns
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X[self.columns]

# Define categorical pipeline
cat_pipe = Pipeline([('selector', ColumnSelector(categorical)),
                     ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
                     ('encoder', OneHotEncoder(handle_unknown='ignore', sparse=False))])

# Define numerical pipeline
num_pipe = Pipeline([('selector', ColumnSelector(numerical))
                     #('imputer', SimpleImputer(strategy='median')),
                     #('scaler', MinMaxScaler())
                    ])

# Combine categorical and numerical pipeline
preprocessor = FeatureUnion(transformer_list=[('cat', cat_pipe),
                                              ('num', num_pipe)])

# Model Training
pipe = Pipeline(steps=[('preprocessor', preprocessor),
              ('model', DecisionTreeRegressor(max_depth=None))])
pipeline_model = pipe.fit(X_train, y_train)

In [ ]:
#คำอธิบาย

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
type(StandardScaler())

In [ ]:
x = [[0,15],[1,-10]]

In [ ]:
type(x)

In [ ]:
scaler_transformer = StandardScaler()

In [ ]:
X = scaler_transformer.fit(x).transform(x)

In [ ]:
X

In [ ]:
y = [0,1]

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rfc = RandomForestClassifier()

In [ ]:
rfc.fit(X,y).predict([[10,100]])